# <center>Lab13: GAN<center/>

111062697 吳律穎

---

1. Implement the Improved WGAN.
2. Train the Improved WGAN on CelebA dataset. Build dataset that read and resize images to 64 x 64 for training.
3. Show a gif of generated samples (at least 8 x 8) to demonstrate the training process and show the best generated sample(s). Please upload to your Google drive and share the link.
4. Draw the loss curve of discriminator and generator during training process into one image.
5. Write a brief report about what you have done.

---

# Import and Parameter setting

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warnings and info
import tensorflow as tf
import tensorflow.keras as keras
import imageio
import moviepy.editor as mpy
import glob
from tqdm import trange


SAMPLE_COL = 8
SAMPLE_ROW = 8
SAMPLE_NUM = SAMPLE_COL * SAMPLE_ROW

IMG_H = 64
IMG_W = 64
IMG_C = 3
IMG_SHAPE = (IMG_H, IMG_W, IMG_C)

BATCH_SIZE = 500
Z_DIM = 128
BUF = 65536

LR = 1e-4
EPOCH = 256

BETA_1 = 0
BETA_2 = 0.9
LAMBDA = 10

IMG_DIR = './imgs/'
if not os.path.exists(IMG_DIR):
    os.makedirs(IMG_DIR)

# Utility function

In [ ]:
def utPuzzle(imgs, row, col, path=None):
    h, w, c = imgs[0].shape
    out = np.zeros((h * row, w * col, c), np.uint8)
    for n, img in enumerate(imgs):
        j, i = divmod(n, col)
        out[j * h : (j + 1) * h, i * w : (i + 1) * w, :] = img
    if path is not None : imageio.imwrite(path, out)
    return out
  
def utMakeGif(imgs, fname, duration):
    n = float(len(imgs)) / duration
    clip = mpy.VideoClip(lambda t : imgs[int(n * t)], duration = duration)
    clip.write_gif(fname, fps = n)

# Dataset

In [ ]:
def map(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=IMG_C)
    img = tf.image.resize(img, (IMG_H, IMG_W))
    img = tf.cast(img, tf.float32)
    img = img / 255.0
    return img

img_name_list = sorted(glob.glob('./dataset/img_align_celeba_png/*.png'))

dataset_train = tf.data.Dataset.from_tensor_slices(img_name_list)\
                               .map(map, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                               .shuffle(BUF)\
                               .batch(BATCH_SIZE, drop_remainder=True)\
                               .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Model

In [ ]:
def GAN(img_shape, z_dim):
    # x-shape
    xh, xw, xc = img_shape
    # z-shape
    zh = xh // 4
    zw = xw // 4
        
    # return Generator and Discriminator
    return keras.Sequential([ # Generator
        keras.layers.Dense(units  =  1024, input_shape = (z_dim,)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Dense(units  =  zh * zw << 8), # zh * zw * 256
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Reshape(target_shape = (zh, zw, 256)),
        keras.layers.Conv2DTranspose(
            filters = 32,
            kernel_size = 5,
            strides = 2,
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(
            filters = xc,
            kernel_size = 5,
            strides = 2,
            padding = "SAME",
            activation = keras.activations.sigmoid
        ),
    ]), keras.Sequential([ # Discriminator
        keras.layers.Conv2D(
            filters = 32,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME",
            input_shape = img_shape,
        ),
        keras.layers.LeakyReLU(),
        keras.layers.Conv2D(
            filters = 128,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        keras.layers.Flatten(),
        keras.layers.Dense(units  =  1024),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        keras.layers.Dense(units  =  1),
    ])

s = tf.random.normal([SAMPLE_NUM, Z_DIM])

In [ ]:
G, D = GAN(IMG_SHAPE, Z_DIM)
optimizer_g = keras.optimizers.Adam(LR, BETA_1, BETA_2)
optimizer_d = keras.optimizers.Adam(LR, BETA_1, BETA_2)

@tf.function
def G_train_step(real_data):
    z = tf.random.normal([BATCH_SIZE, Z_DIM])

    with tf.GradientTape() as tape:
        fake_data = G(z, training=True)

        D_f = D(fake_data, training=True)
        D_r = D(real_data, training=True)

        loss_g = tf.reduce_mean(-D_f)

        epsilon = tf.random.uniform([BATCH_SIZE, 1, 1, 1])
        x_hat = epsilon * real_data + (1 - epsilon) * fake_data
        gradient_x_hat = tf.gradients(D(x_hat, training=True), x_hat)[0]
        l2_norm = (tf.sqrt(tf.reduce_sum(gradient_x_hat ** 2, axis=[1, 2, 3])) - 1.) ** 2
        loss_d = tf.reduce_mean(D_f - D_r + LAMBDA * l2_norm)

    gradient_g = tape.gradient(loss_g, G.trainable_variables)
    optimizer_g.apply_gradients(zip(gradient_g, G.trainable_variables))

    return loss_g, loss_d


@tf.function
def D_train_step(real_data):
    z = tf.random.normal([BATCH_SIZE, Z_DIM])

    with tf.GradientTape() as tape:
        fake_data = G(z, training=True)

        D_f = D(fake_data, training=True)
        D_r = D(real_data, training=True)

        loss_g = tf.reduce_mean(-D_f)

        epsilon = tf.random.uniform([BATCH_SIZE, 1, 1, 1])
        x_hat = epsilon * real_data + (1 - epsilon) * fake_data
        gradient_x_hat = tf.gradients(D(x_hat, training=True), x_hat)[0]
        l2_norm = (tf.sqrt(tf.reduce_sum(gradient_x_hat ** 2, axis=[1, 2, 3])) - 1.) ** 2
        
        loss_d = tf.reduce_mean(D_f - D_r + LAMBDA * l2_norm)

    gradient_d = tape.gradient(loss_d, D.trainable_variables)
    optimizer_d.apply_gradients(zip(gradient_d, D.trainable_variables))

    return loss_g, loss_d

In [ ]:
train_step = (
    D_train_step,
    D_train_step,
    D_train_step,
    D_train_step,
    D_train_step,
    G_train_step
)

num_critic = len(train_step)

# Training

In [ ]:
g_loss_list = [None] * EPOCH  # record loss of g for each epoch
d_loss_list = [None] * EPOCH  # record loss of d for each epoch
sample_list = [None] * EPOCH  # record sample images for each epoch

sample_z = tf.random.normal([SAMPLE_NUM, Z_DIM])

critic = 0
STEP = len(img_name_list) // BATCH_SIZE

pbar = trange(EPOCH, unit='epoch')
for epoch in pbar:
    loss_g_t = 0.0
    loss_d_t = 0.0
    for real_data in dataset_train:
        loss_g, loss_d = train_step[critic](real_data)
        critic = critic + 1 if critic + 1 < num_critic else 0
        loss_g_t += loss_g.numpy()
        loss_d_t += loss_d.numpy()

    g_loss_list[epoch] = loss_g_t / STEP
    d_loss_list[epoch] = loss_d_t / STEP
    pbar.set_postfix({'g_loss': loss_g_t / STEP, 'd_loss': loss_d_t / STEP})

    out = G(sample_z, training=False)
    img = utPuzzle(
        (out * 255.0).numpy().astype(np.uint8),
        SAMPLE_COL,
        SAMPLE_ROW,
        f'{IMG_DIR}WGAN-Improved_{epoch:04d}.png'
    )
    sample_list[epoch] = img
    if (epoch + 1) % 25 == 0:
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Epoch {epoch:d}')
        plt.show()

# Evaluate 

In [ ]:
utMakeGif(np.array(sample_list), f'{IMG_DIR}WGAN-Improved.gif', duration=2)

In [ ]:
plt.plot(range(EPOCH), g_loss_list, color='red', label='Generator Loss')
plt.plot(range(EPOCH), d_loss_list, color='blue', label='Discriminator Loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('WGAN-Improved Training Loss')
plt.tight_layout()
plt.savefig(f'{IMG_DIR}WGAN-Improved_Training_Loss.png', facecolor='white')
plt.show()

# Report

Loss 是依照 Lab13 notebook 給的 pseudo code 來更改助教 loss 的訂法， hyperparameter 也是依照 pseudo code 上方給的數值來賦值。 而 model architecture 則是直接用 lab13 notebook 中給的架構。 